<a href="https://colab.research.google.com/github/kunwarsharik/SharikProjects/blob/master/lang_convrt_ger_to_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://www.manythings.org/anki/deu-eng.zip

--2020-01-31 16:38:23--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:3037::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7747747 (7.4M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   7.39M  4.52MB/s    in 1.6s    

2020-01-31 16:38:25 (4.52 MB/s) - ‘deu-eng.zip’ saved [7747747/7747747]



In [0]:
import re,string
from pickle import dump,load
from unicodedata import normalize
from numpy import array,argmax
from numpy.random import rand,shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential,load_model
from keras.layers import LSTM, Dense,Embedding,RepeatVector,TimeDistributed,Dropout
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import corpus_bleu

In [0]:
ls

deu-eng.zip  sample_data/


In [0]:
!unzip 'deu-eng.zip'

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [0]:
file=open('deu.txt',mode='rt',encoding='utf-8')
doc=file.read()
file.close()

In [0]:
lines=doc.strip().split('\n')
pairs=[line.split('\t') for line in lines]

In [0]:
pairs[1]

['Hi.',
 'Grüß Gott!',
 'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)']

In [0]:
cleaned=[]
re_print=re.compile('[^%s]'%re.escape(string.printable))
table=str.maketrans('', '', string.punctuation)
for p in range(len(pairs)-1):
    pair=pairs[p]
    clean_pair=[]
    for l in pair:
        l=normalize('NFD',l).encode('ascii','ignore')#retun in binary
        l= l.decode('UTF-8')#convert to string
        l = l.split()#line to list
        l = [word.lower() for word in l]#each list  ele in lowercase
        l = [word.translate(table) for word in l]#punctuation remove
        l = [re_print.sub('', w) for w in l]#non printable char remove
        l = [word for word in l if word.isalpha()]#word with number remove
        clean_pair.append(' '.join(l))
    cleaned.append(clean_pair)    

In [0]:
cleaned=array(cleaned)

In [0]:
for i in range(10):
    print(cleaned[i,0]+str("=>")+cleaned[i,1])

hi=>hallo
hi=>gru gott
run=>lauf
wow=>potzdonner
wow=>donnerwetter
fire=>feuer
help=>hilfe
help=>zu hulf
stop=>stopp
wait=>warte


In [0]:
cleaned.shape

(204573, 3)

In [0]:
n_sentences = 53000
dataset=cleaned[:n_sentences,:2]
train, test = dataset[:50000], dataset[50000:52000]

In [0]:
eng_tok=Tokenizer()
eng_tok.fit_on_texts(dataset[:,0])
ger_tok=Tokenizer()
ger_tok.fit_on_texts(dataset[:,1])
me=max(len(line.split()) for line in dataset[:,0])
mg=max(len(line.split()) for line in dataset[:,1])

In [0]:
me,mg

(7, 17)

In [0]:
eng_vocab_size=len(eng_tok.word_index)+1
ger_vocab_size=len(ger_tok.word_index)+1

In [0]:
eng_vocab_size,ger_vocab_size

(6435, 10679)

In [0]:
k1=eng_tok.texts_to_sequences(train[:,0])
fe=pad_sequences(k1,maxlen=me,padding='post')

k2=ger_tok.texts_to_sequences(train[:,1])
fg=pad_sequences(k2,maxlen=mg,padding='post')

In [0]:
k1=eng_tok.texts_to_sequences(test[:,0])
fe_t=pad_sequences(k1,maxlen=me,padding='post')

k2=ger_tok.texts_to_sequences(test[:,1])
fg_t=pad_sequences(k2,maxlen=mg,padding='post')

In [0]:
fe_t.shape,fe.shape,fg.shape,fg_t.shape

((2000, 7), (50000, 7), (50000, 17), (2000, 17))

In [0]:
#one hot key encode sequnce for output
y,y_t=[],[]
for seq in fe:
    encoded=to_categorical(seq,num_classes=eng_vocab_size)
    y.append(encoded)
for seq in fe_t:
    encoded=to_categorical(seq,num_classes=eng_vocab_size)
    y_t.append(encoded)
y,y_t = array(y),array(y_t)
y = y.reshape(fe.shape[0], fe.shape[1], eng_vocab_size)
y_t = y_t.reshape(fe_t.shape[0], fe_t.shape[1], eng_vocab_size)

In [0]:
y.shape,y_t.shape

((50000, 7, 6435), (2000, 7, 6435))

In [24]:
model = Sequential()
model.add(Embedding(ger_vocab_size, 1000, input_length=mg, mask_zero=True))
model.add(LSTM(700, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(1000))
model.add(RepeatVector(me))
model.add(LSTM(1000, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(700, return_sequences=True))
model.add(TimeDistributed(Dense(eng_vocab_size, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')
plot_model(model, to_file='model.png', show_shapes=True)
filename = 'gertoeng.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(fg, y, epochs=100, batch_size=64, validation_data=(fg_t, y_t), callbacks=[checkpoint], verbose=2)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





Train on 50000 samples, validate on 2000 samples
Epoch 1/100





 - 103s - loss: 3.5458 - val_loss: 3.7767

Epoch 00001: val_loss improved from inf to 3.77672, saving model to gertoeng.h5
Epoch 2/100
 - 90s - loss: 2.7389 - val_loss: 3.5417

Epoch 00002: val_loss improved from 3.77672 to 3.54171, saving model to gertoeng.h5
Epoch 3/100
 - 90s - loss: 2.4157 - val_loss: 3.3442

Epoch 00003: val_loss improved from 3.54171 to 3.34422, saving model to gertoeng.h5
Epoch 4/100
 - 90s - loss: 2.1711 - val_loss: 3.2448

Epoch 00004: val_loss improved from 3.34422 to 3.24482, saving model to gertoeng.h5
Epoch 5/100
 - 91s - loss: 1.9671 - val_loss: 3.1415

Epoch 00005: val_loss improved from 3.24482 to 3.14152, saving model to gertoeng.h5
Epoch 6/100
 - 90s - loss: 1.7764 - val_l

In [25]:
for s2 in range(10):
    sample=fg_t[s2]
    pred=model.predict(sample.reshape(1,len(fg_t[0])))
    x=[argmax(v) for v in pred[0]]
    target=[]
    for w,i in eng_tok.word_index.items():
        if i in x:
            target.append(w)
    res=' '.join(target)
    print("GERMAN : ",test[:,1][s2],"\nDECODED : ",res, "\nACTUAL : ", test[:,0][s2])

GERMAN :  er hat mit dem rauchen aufgehort 
DECODED :  he stopped smoking 
ACTUAL :  he has stopped smoking
GERMAN :  er hat drei tochter 
DECODED :  he has three daughters 
ACTUAL :  he has three daughters
GERMAN :  er hat sehr kurze haare 
DECODED :  he has hair blue large 
ACTUAL :  he has very short hair
GERMAN :  er ist noch nicht erschienen 
DECODED :  is to he from 
ACTUAL :  he hasnt appeared yet
GERMAN :  er versteckte sich hinter der tur 
DECODED :  the he off ran legs 
ACTUAL :  he hid behind the door
GERMAN :  er stritt es sofort ab 
DECODED :  the he denied mobile 
ACTUAL :  he instantly denied it
GERMAN :  er ist ein fahiger anwalt 
DECODED :  is a he artist tallest 
ACTUAL :  he is a capable lawyer
GERMAN :  er ist ein guter zimmermann 
DECODED :  is a he good heart 
ACTUAL :  he is a good carpenter
GERMAN :  er ist ein guter violinist 
DECODED :  a good hes man 
ACTUAL :  he is a good violinist
GERMAN :  er ist ein echter gentleman 
DECODED :  a hes real gentleman 
ACTU